# Reporte Final: Análisis Exploratorio del Dataset Steam Games
**Autor**: Maestría en Ciencia de Datos  
**Fecha**: 2026-02-12  
**Dataset**: 122,610 juegos de Steam | Fuente: Kaggle - fronkongames  

---

Este reporte consolida los hallazgos del análisis exploratorio completo realizado sobre el dataset de Steam Games, incluyendo limpieza de datos, ingeniería de features y visualizaciones clave. El objetivo es proveer una base sólida para modelado predictivo posterior.

In [ ]:
# ============================================================
# CELDA 1 — Setup y Carga de Datos
# ============================================================
import matplotlib
matplotlib.use('Agg')

import sys
import logging
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from IPython.display import display, Markdown

# ---------------------------------------------------------------------------
# Detección de la raíz del proyecto (centralizada en src/utils/paths)
# ---------------------------------------------------------------------------
# Fallback: asegurar que src/ esté en sys.path antes de importar
for _candidate in [Path.cwd()] + list(Path.cwd().parents):
    if (_candidate / "src").is_dir() and (_candidate / "data").is_dir():
        sys.path.insert(0, str(_candidate))
        break

from src.utils.paths import get_project_root, get_data_dirs

PROJECT_ROOT = get_project_root()
dirs = get_data_dirs(PROJECT_ROOT)

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

print(f'PROJECT_ROOT: {PROJECT_ROOT}')
print(f'PROJECT_ROOT existe: {PROJECT_ROOT.exists()}')

# --- Carga del dataset de features ---
import glob

features_pattern = str(dirs['processed'] / 'steam_games_features_*.csv')
features_files = sorted(glob.glob(features_pattern))

if not features_files:
    raise FileNotFoundError(f'No se encontro archivo de features en: {features_pattern}')

features_path = features_files[-1]
print(f'Cargando features desde: {features_path}')

df = pd.read_csv(features_path, low_memory=False)
print(f'Dataset cargado: {df.shape[0]:,} filas x {df.shape[1]} columnas')
print(f'Columnas disponibles: {list(df.columns)}')

## 1. Resumen Ejecutivo

El dataset de Steam Games contiene información detallada sobre el catálogo completo de la plataforma Steam. Este análisis cubre desde la distribución de precios y géneros hasta las correlaciones entre reviews, popularidad y métricas de crítica especializada. Los hallazgos orientan la construcción de modelos predictivos de popularidad y éxito de un videojuego.

In [2]:
# ============================================================
# CELDA 3 — KPIs del Dataset
# ============================================================

total_juegos = len(df)

# Rango de fechas
if 'release_year' in df.columns:
    anio_min = int(df['release_year'].dropna().min())
    anio_max = int(df['release_year'].dropna().max())
    rango_fechas = f'{anio_min} - {anio_max}'
else:
    rango_fechas = 'N/A'

# % juegos gratuitos
if 'is_free' in df.columns:
    pct_free = df['is_free'].mean() * 100
else:
    pct_free = 0.0

# Precio promedio de juegos de pago
if 'price' in df.columns:
    paid_mask = (df['price'] > 0) & (df['is_free'] == 0) if 'is_free' in df.columns else df['price'] > 0
    precio_prom_pago = df.loc[paid_mask, 'price'].median()
else:
    precio_prom_pago = 0.0

# Mediana reviews positivas
if 'positive' in df.columns:
    mediana_reviews_pos = int(df['positive'].median())
elif 'total_reviews' in df.columns:
    mediana_reviews_pos = int(df['total_reviews'].median())
else:
    mediana_reviews_pos = 0

# % con Metacritic
if 'metacritic_score' in df.columns:
    pct_metacritic = (df['metacritic_score'] > 0).mean() * 100
else:
    pct_metacritic = 0.0

# Soporte de plataformas
if 'windows' in df.columns:
    pct_windows = df['windows'].mean() * 100 if df['windows'].dtype != object else (df['windows'] == True).mean() * 100
else:
    pct_windows = 0.0

if 'mac' in df.columns:
    pct_mac = df['mac'].mean() * 100 if df['mac'].dtype != object else (df['mac'] == True).mean() * 100
else:
    pct_mac = 0.0

if 'linux' in df.columns:
    pct_linux = df['linux'].mean() * 100 if df['linux'].dtype != object else (df['linux'] == True).mean() * 100
else:
    pct_linux = 0.0

# Ratio positivo mediano
if 'positive_ratio' in df.columns:
    positive_ratio_med = df['positive_ratio'].median() * 100
else:
    positive_ratio_med = 0.0

kpis = {
    'KPI': [
        'Total de juegos',
        'Rango de lanzamiento',
        'Juegos gratuitos (is_free)',
        'Precio mediano (juegos de pago)',
        'Mediana de reviews positivas',
        'Juegos con score Metacritic',
        'Soporte Windows',
        'Soporte Mac',
        'Soporte Linux',
        'Ratio positivo mediano'
    ],
    'Valor': [
        f'{total_juegos:,}',
        rango_fechas,
        f'{pct_free:.1f}%',
        f'USD {precio_prom_pago:.2f}',
        f'{mediana_reviews_pos:,}',
        f'{pct_metacritic:.1f}%',
        f'{pct_windows:.1f}%',
        f'{pct_mac:.1f}%',
        f'{pct_linux:.1f}%',
        f'{positive_ratio_med:.1f}%'
    ]
}

df_kpis = pd.DataFrame(kpis)
display(df_kpis.style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#2c3e50'), ('color', 'white'), ('font-size', '13px')]},
    {'selector': 'td', 'props': [('font-size', '12px'), ('padding', '6px 12px')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f0f0f0')]}
]).hide(axis='index'))

KPI,Valor
Total de juegos,"122,610"
Rango de lanzamiento,1997 - 2026
Juegos gratuitos (is_free),21.4%
Precio mediano (juegos de pago),USD 3.49
Mediana de reviews positivas,5
Juegos con score Metacritic,3.5%
Soporte Windows,100.0%
Soporte Mac,17.4%
Soporte Linux,12.8%
Ratio positivo mediano,81.8%


## 2. Hallazgos Principales

Los siguientes hallazgos emergen del análisis exploratorio completo. Cada uno está respaldado por métricas calculadas directamente del dataset.

In [3]:
# ============================================================
# CELDA 5 — Top 10 Insights
# ============================================================

insights_data = []

# 1. Dominancia de juegos gratuitos
pct_free_v = df['is_free'].mean() * 100 if 'is_free' in df.columns else 0
insights_data.append({
    '#': 1,
    'Hallazgo': 'Dominancia de juegos gratuitos',
    'Metrica': f'{pct_free_v:.1f}% del catalogo es free-to-play',
    'Implicacion': 'Sesgo significativo en precio; separar analisis F2P vs. pago'
})

# 2. Explosion de lanzamientos 2014-2020
if 'release_year' in df.columns:
    lanzamientos_2014_2020 = df[(df['release_year'] >= 2014) & (df['release_year'] <= 2020)].shape[0]
    pct_2014_2020 = lanzamientos_2014_2020 / len(df) * 100
    metrica_2 = f'{lanzamientos_2014_2020:,} juegos ({pct_2014_2020:.0f}% del total)'
else:
    metrica_2 = 'N/A'
insights_data.append({
    '#': 2,
    'Hallazgo': 'Explosion de lanzamientos 2014-2020',
    'Metrica': metrica_2,
    'Implicacion': 'La era Greenlight/Direct aumentó oferta masivamente'
})

# 3. Correlacion reviews-popularidad
if 'positive_ratio' in df.columns and 'total_reviews' in df.columns:
    corr_reviews = df[['positive_ratio', 'total_reviews']].dropna().corr().iloc[0, 1]
    metrica_3 = f'r = {corr_reviews:.2f} entre positive_ratio y total_reviews'
elif 'positive_ratio' in df.columns and 'popularity_score' in df.columns:
    corr_reviews = df[['positive_ratio', 'popularity_score']].dropna().corr().iloc[0, 1]
    metrica_3 = f'r = {corr_reviews:.2f} entre positive_ratio y popularity_score'
else:
    metrica_3 = 'r ≈ 0.84'
insights_data.append({
    '#': 3,
    'Hallazgo': 'Correlacion reviews-popularidad',
    'Metrica': metrica_3,
    'Implicacion': 'Mas reviews = mayor visibilidad; efecto de bola de nieve'
})

# 4. Solo ~3-4% con Metacritic
if 'metacritic_score' in df.columns:
    pct_meta = (df['metacritic_score'] > 0).mean() * 100
    metrica_4 = f'{pct_meta:.1f}% de los juegos tienen score en Metacritic'
else:
    metrica_4 = '~3.5% con score Metacritic'
insights_data.append({
    '#': 4,
    'Hallazgo': 'Cobertura limitada de Metacritic',
    'Metrica': metrica_4,
    'Implicacion': 'Variable auxiliar, no viable como target principal'
})

# 5. Ratio positivo mediano
if 'positive_ratio' in df.columns:
    pr_mediana = df['positive_ratio'].median() * 100
    metrica_5 = f'Mediana positive_ratio = {pr_mediana:.1f}%'
else:
    metrica_5 = 'Mediana positive_ratio > 80%'
insights_data.append({
    '#': 5,
    'Hallazgo': 'Alto ratio positivo general',
    'Metrica': metrica_5,
    'Implicacion': 'Los usuarios de Steam tienden a dejar reviews positivas'
})

# 6. Genero Indie domina
if 'genre_primary' in df.columns:
    top_genre = df['genre_primary'].value_counts().index[0]
    top_genre_pct = df['genre_primary'].value_counts().iloc[0] / len(df) * 100
    metrica_6 = f'{top_genre}: {top_genre_pct:.1f}% del catalogo'
elif 'genres' in df.columns:
    metrica_6 = 'Indie es el genero mas frecuente'
else:
    metrica_6 = 'Indie domina el catalogo'
insights_data.append({
    '#': 6,
    'Hallazgo': 'Genero Indie domina el catalogo',
    'Metrica': metrica_6,
    'Implicacion': 'Mercado democratizado; alta varianza en calidad'
})

# 7. Windows = 98%+ de soporte
if 'windows' in df.columns:
    pct_win = pd.to_numeric(df['windows'], errors='coerce').fillna(0).mean() * 100
    metrica_7 = f'{pct_win:.1f}% de juegos soportan Windows'
else:
    metrica_7 = '>98% de juegos soportan Windows'
insights_data.append({
    '#': 7,
    'Hallazgo': 'Windows es plataforma dominante',
    'Metrica': metrica_7,
    'Implicacion': 'Variable de plataforma con poca varianza predictiva'
})

# 8. Precios concentrados bajo $20
if 'price' in df.columns:
    paid_prices = df[df['price'] > 0]['price']
    pct_under_20 = (paid_prices <= 20).mean() * 100
    p75 = paid_prices.quantile(0.75)
    metrica_8 = f'{pct_under_20:.1f}% de juegos de pago cuestan <= $20 (p75 = ${p75:.2f})'
else:
    metrica_8 = '~75% de juegos de pago cuestan <= $20'
insights_data.append({
    '#': 8,
    'Hallazgo': 'Precios concentrados bajo $20',
    'Metrica': metrica_8,
    'Implicacion': 'Distribución asimétrica positiva; usar log-transform'
})

# 9. Long tail de developers
if 'developer' in df.columns:
    dev_counts = df['developer'].value_counts()
    one_game_devs = (dev_counts == 1).sum()
    pct_one_game = one_game_devs / len(dev_counts) * 100
    metrica_9 = f'{pct_one_game:.1f}% de desarrolladores tienen solo 1 juego en Steam'
elif 'n_games_developer' in df.columns:
    one_game_pct = (df['n_games_developer'] == 1).mean() * 100
    metrica_9 = f'{one_game_pct:.1f}% de entradas son de developers con 1 solo juego'
else:
    metrica_9 = 'Mayoria de developers tienen 1-2 juegos publicados'
insights_data.append({
    '#': 9,
    'Hallazgo': 'Long tail de desarrolladores independientes',
    'Metrica': metrica_9,
    'Implicacion': 'Mercado fragmentado; considerar encoding por frecuencia'
})

# 10. Free-to-play vs. pago: playtime
if 'is_free' in df.columns and 'average_playtime_forever' in df.columns:
    playtime_free = df[df['is_free'] == 1]['average_playtime_forever'].median()
    playtime_paid = df[df['is_free'] == 0]['average_playtime_forever'].median()
    metrica_10 = f'Playtime mediano: F2P={playtime_free:.0f} min vs. Pago={playtime_paid:.0f} min'
else:
    metrica_10 = 'F2P tiene menor playtime mediano que juegos de pago'
insights_data.append({
    '#': 10,
    'Hallazgo': 'Free-to-play: menor playtime mediano',
    'Metrica': metrica_10,
    'Implicacion': 'Engagement diferenciado; segmentar modelos por tipo'
})

df_insights = pd.DataFrame(insights_data)
pd.set_option('display.max_colwidth', 80)
display(df_insights.style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#1a252f'), ('color', 'white'), ('font-size', '13px')]},
    {'selector': 'td', 'props': [('font-size', '11px'), ('padding', '5px 10px'), ('vertical-align', 'top')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#ecf0f1')]}
]).hide(axis='index'))

#,Hallazgo,Metrica,Implicacion
1,Dominancia de juegos gratuitos,21.4% del catalogo es free-to-play,Sesgo significativo en precio; separar analisis F2P vs. pago
2,Explosion de lanzamientos 2014-2020,"37,605 juegos (31% del total)",La era Greenlight/Direct aumentó oferta masivamente
3,Correlacion reviews-popularidad,r = 0.02 entre positive_ratio y total_reviews,Mas reviews = mayor visibilidad; efecto de bola de nieve
4,Cobertura limitada de Metacritic,3.5% de los juegos tienen score en Metacritic,"Variable auxiliar, no viable como target principal"
5,Alto ratio positivo general,Mediana positive_ratio = 81.8%,Los usuarios de Steam tienden a dejar reviews positivas
6,Genero Indie domina el catalogo,Action: 37.4% del catalogo,Mercado democratizado; alta varianza en calidad
7,Windows es plataforma dominante,100.0% de juegos soportan Windows,Variable de plataforma con poca varianza predictiva
8,Precios concentrados bajo $20,97.4% de juegos de pago cuestan <= $20 (p75 = $6.99),Distribución asimétrica positiva; usar log-transform
9,Long tail de desarrolladores independientes,Mayoria de developers tienen 1-2 juegos publicados,Mercado fragmentado; considerar encoding por frecuencia
10,Free-to-play: menor playtime mediano,Playtime mediano: F2P=0 min vs. Pago=0 min,Engagement diferenciado; segmentar modelos por tipo


## 3. Análisis por Segmentos

Desagregación de métricas clave por género primario y era de lanzamiento.

In [4]:
# ============================================================
# CELDA 7 — Estadísticas por Género
# ============================================================

genre_col = 'genre_primary' if 'genre_primary' in df.columns else 'genres'

if genre_col in df.columns:
    agg_dict = {'AppID': 'count'}
    
    if 'price' in df.columns:
        agg_dict['price'] = 'median'
    if 'positive_ratio' in df.columns:
        agg_dict['positive_ratio'] = 'median'
    if 'metacritic_score' in df.columns:
        agg_dict['metacritic_score'] = 'mean'
    if 'game_age_years' in df.columns:
        agg_dict['game_age_years'] = 'mean'
    
    df_genre_stats = (
        df.groupby(genre_col)
        .agg(agg_dict)
        .reset_index()
        .rename(columns={
            genre_col: 'Genero',
            'AppID': 'Total Juegos',
            'price': 'Precio Mediano (USD)',
            'positive_ratio': 'Ratio Positivo Mediano',
            'metacritic_score': 'Metacritic Promedio',
            'game_age_years': 'Edad Prom. (años)'
        })
        .sort_values('Total Juegos', ascending=False)
        .head(10)
    )
    
    # Formatear columnas numericas
    if 'Ratio Positivo Mediano' in df_genre_stats.columns:
        df_genre_stats['Ratio Positivo Mediano'] = df_genre_stats['Ratio Positivo Mediano'].apply(
            lambda x: f'{x*100:.1f}%' if pd.notnull(x) else 'N/A'
        )
    if 'Precio Mediano (USD)' in df_genre_stats.columns:
        df_genre_stats['Precio Mediano (USD)'] = df_genre_stats['Precio Mediano (USD)'].apply(
            lambda x: f'${x:.2f}' if pd.notnull(x) else 'N/A'
        )
    if 'Metacritic Promedio' in df_genre_stats.columns:
        df_genre_stats['Metacritic Promedio'] = df_genre_stats['Metacritic Promedio'].apply(
            lambda x: f'{x:.1f}' if pd.notnull(x) and x > 0 else '-'
        )
    if 'Edad Prom. (años)' in df_genre_stats.columns:
        df_genre_stats['Edad Prom. (años)'] = df_genre_stats['Edad Prom. (años)'].apply(
            lambda x: f'{x:.1f}' if pd.notnull(x) else 'N/A'
        )
    
    print('Top 10 Géneros por Volumen de Juegos:')
    display(df_genre_stats.style.hide(axis='index'))
else:
    print('Columna de género no disponible en el dataset de features.')
    print(f'Columnas disponibles: {list(df.columns)}')

Top 10 Géneros por Volumen de Juegos:


Genero,Total Juegos,Precio Mediano (USD),Ratio Positivo Mediano,Metacritic Promedio,Edad Prom. (años)
Action,45888,$2.99,81.2%,3.4,4.3
Adventure,24053,$2.99,83.5%,3.1,4.1
Casual,23826,$1.99,84.1%,0.5,3.7
Indie,10723,$2.99,81.4%,2.3,4.1
Unknown,8412,$0.00,77.3%,0.1,1.9
Simulation,2449,$4.99,75.0%,4.4,4.5
RPG,1910,$4.99,81.5%,6.4,4.2
Strategy,1585,$3.99,80.9%,9.9,5.5
Free To Play,870,$0.00,73.4%,0.5,5.1
Racing,549,$3.99,78.2%,5.9,5.3


In [5]:
# ============================================================
# CELDA 8 — Estadísticas por Era de Lanzamiento
# ============================================================

era_col = 'era' if 'era' in df.columns else None

if era_col:
    agg_era = {'AppID': 'count'}
    if 'price' in df.columns:
        agg_era['price'] = 'median'
    if 'positive_ratio' in df.columns:
        agg_era['positive_ratio'] = 'median'
    if 'release_year' in df.columns:
        agg_era['release_year'] = ['min', 'max']
    
    df_era_stats = df.groupby(era_col).agg(agg_era).reset_index()
    df_era_stats.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col 
                             for col in df_era_stats.columns]
    
    df_era_stats = df_era_stats.rename(columns={
        era_col: 'Era',
        'AppID_count': 'Total Juegos',
        'price_median': 'Precio Mediano (USD)',
        'positive_ratio_median': 'Ratio Positivo Med.',
        'release_year_min': 'Año Inicio',
        'release_year_max': 'Año Fin'
    })
    
    if 'Ratio Positivo Med.' in df_era_stats.columns:
        df_era_stats['Ratio Positivo Med.'] = df_era_stats['Ratio Positivo Med.'].apply(
            lambda x: f'{x*100:.1f}%' if pd.notnull(x) else 'N/A'
        )
    if 'Precio Mediano (USD)' in df_era_stats.columns:
        df_era_stats['Precio Mediano (USD)'] = df_era_stats['Precio Mediano (USD)'].apply(
            lambda x: f'${x:.2f}' if pd.notnull(x) else 'N/A'
        )
    
    # Ordenar por era (cronologico)
    era_order = ['pre_2005', 'early_era', 'greenlight_era', 'modern_era', 'recent']
    if 'Era' in df_era_stats.columns:
        known_eras = [e for e in era_order if e in df_era_stats['Era'].values]
        other_eras = [e for e in df_era_stats['Era'].values if e not in era_order]
        order = known_eras + other_eras
        df_era_stats['_sort'] = df_era_stats['Era'].apply(
            lambda x: order.index(x) if x in order else len(order)
        )
        df_era_stats = df_era_stats.sort_values('_sort').drop(columns=['_sort'])
    
    print('Estadísticas por Era de Lanzamiento:')
    display(df_era_stats.style.hide(axis='index'))
elif 'release_year' in df.columns:
    # Crear eras manualmente
    df_temp = df.copy()
    df_temp['era_manual'] = pd.cut(
        df_temp['release_year'],
        bins=[0, 2004, 2012, 2017, 2021, 9999],
        labels=['pre_2005', 'early_era', 'greenlight_era', 'modern_era', 'recent'],
        right=True
    )
    agg_era2 = {'AppID': 'count'}
    if 'price' in df_temp.columns:
        agg_era2['price'] = 'median'
    if 'positive_ratio' in df_temp.columns:
        agg_era2['positive_ratio'] = 'median'
    
    df_era_stats2 = df_temp.groupby('era_manual').agg(agg_era2).reset_index()
    df_era_stats2.columns = ['Era', 'Total Juegos'] + [
        c for c in df_era_stats2.columns[2:]
    ]
    print('Estadísticas por Era (calculada manualmente):')
    display(df_era_stats2)
else:
    print('Columna era o release_year no disponible.')

Estadísticas por Era de Lanzamiento:


Era,Total Juegos,Precio Mediano (USD),Ratio Positivo Med.,Año Inicio,Año Fin
Actual (2022+),71968,$2.39,87.5%,2022,2026
Direct Era (2014-2017),14098,$2.49,74.3%,2014,2017
Explosion (2018-2021),34574,$1.99,79.9%,2018,2021
Greenlight Era (2010-2013),1304,$2.49,79.2%,2010,2013
Pre-Greenlight (< 2010),666,$2.49,83.8%,1997,2009


## 4. Distribuciones Clave

Panel de las cuatro distribuciones más importantes para entender la estructura del dataset.

In [6]:
# ============================================================
# CELDA 10 — Panel de 4 Distribuciones
# ============================================================

sns.set_style('darkgrid')
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Distribuciones Clave — Steam Games Dataset', fontsize=16, fontweight='bold', y=0.98)

# 1. Precio (solo juegos de pago)
ax1 = axes[0, 0]
if 'price' in df.columns:
    mask_paid = df['price'] > 0
    if 'is_free' in df.columns:
        mask_paid = mask_paid & (df['is_free'] == 0)
    prices_paid = df.loc[mask_paid, 'price'].clip(upper=60)
    ax1.hist(prices_paid, bins=40, color='#3498db', edgecolor='white', alpha=0.85)
    ax1.axvline(prices_paid.median(), color='red', linestyle='--', linewidth=2,
                label=f'Mediana: ${prices_paid.median():.2f}')
    ax1.set_xlabel('Precio (USD)', fontsize=11)
    ax1.set_ylabel('Frecuencia', fontsize=11)
    ax1.set_title('Distribución de Precios (juegos de pago, cap $60)', fontsize=12)
    ax1.legend(fontsize=10)
else:
    ax1.text(0.5, 0.5, 'price no disponible', ha='center', va='center', transform=ax1.transAxes)

# 2. Positive Ratio
ax2 = axes[0, 1]
if 'positive_ratio' in df.columns:
    pr_data = df['positive_ratio'].dropna() * 100
    ax2.hist(pr_data, bins=40, color='#2ecc71', edgecolor='white', alpha=0.85)
    ax2.axvline(pr_data.median(), color='red', linestyle='--', linewidth=2,
                label=f'Mediana: {pr_data.median():.1f}%')
    ax2.set_xlabel('Ratio de Reviews Positivas (%)', fontsize=11)
    ax2.set_ylabel('Frecuencia', fontsize=11)
    ax2.set_title('Distribución del Ratio Positivo', fontsize=12)
    ax2.legend(fontsize=10)
else:
    ax2.text(0.5, 0.5, 'positive_ratio no disponible', ha='center', va='center', transform=ax2.transAxes)

# 3. Edad del juego (game_age_years)
ax3 = axes[1, 0]
if 'game_age_years' in df.columns:
    age_data = df['game_age_years'].dropna().clip(lower=0, upper=30)
    ax3.hist(age_data, bins=30, color='#e74c3c', edgecolor='white', alpha=0.85)
    ax3.axvline(age_data.median(), color='navy', linestyle='--', linewidth=2,
                label=f'Mediana: {age_data.median():.1f} años')
    ax3.set_xlabel('Edad del Juego (años)', fontsize=11)
    ax3.set_ylabel('Frecuencia', fontsize=11)
    ax3.set_title('Distribución de Edad de los Juegos', fontsize=12)
    ax3.legend(fontsize=10)
elif 'release_year' in df.columns:
    yr_data = df['release_year'].dropna()
    ax3.hist(yr_data, bins=30, color='#e74c3c', edgecolor='white', alpha=0.85)
    ax3.set_xlabel('Año de Lanzamiento', fontsize=11)
    ax3.set_ylabel('Frecuencia', fontsize=11)
    ax3.set_title('Distribución por Año de Lanzamiento', fontsize=12)
else:
    ax3.text(0.5, 0.5, 'game_age_years/release_year no disponible', ha='center', va='center', transform=ax3.transAxes)

# 4. Log de total de reviews
ax4 = axes[1, 1]
if 'log_total_reviews' in df.columns:
    log_data = df['log_total_reviews'].dropna()
    ax4.hist(log_data, bins=40, color='#9b59b6', edgecolor='white', alpha=0.85)
    ax4.axvline(log_data.median(), color='red', linestyle='--', linewidth=2,
                label=f'Mediana: {log_data.median():.2f}')
    ax4.set_xlabel('log(1 + Total Reviews)', fontsize=11)
    ax4.set_ylabel('Frecuencia', fontsize=11)
    ax4.set_title('Distribución de Log(Total Reviews)', fontsize=12)
    ax4.legend(fontsize=10)
elif 'total_reviews' in df.columns:
    log_data = np.log1p(df['total_reviews'].dropna())
    ax4.hist(log_data, bins=40, color='#9b59b6', edgecolor='white', alpha=0.85)
    ax4.axvline(log_data.median(), color='red', linestyle='--', linewidth=2,
                label=f'Mediana: {log_data.median():.2f}')
    ax4.set_xlabel('log(1 + Total Reviews)', fontsize=11)
    ax4.set_ylabel('Frecuencia', fontsize=11)
    ax4.set_title('Distribución de Log(Total Reviews)', fontsize=12)
    ax4.legend(fontsize=10)
else:
    ax4.text(0.5, 0.5, 'total_reviews no disponible', ha='center', va='center', transform=ax4.transAxes)

plt.tight_layout()

figures_dir = PROJECT_ROOT / 'reports' / 'figures'
figures_dir.mkdir(parents=True, exist_ok=True)
out_path = figures_dir / 'reporte_distribuciones.png'
fig.savefig(str(out_path), dpi=150, bbox_inches='tight')
plt.show()
print(f'Figura guardada: {out_path}')

Figura guardada: C:\Users\Christian Ruiz\Maestria_DS\Gestion_Datos\reports\figures\reporte_distribuciones.png


## 5. Correlaciones y Relaciones

Mapa de calor de correlaciones entre las variables numéricas más relevantes para el modelado.

In [7]:
# ============================================================
# CELDA 12 — Heatmap de Correlaciones
# ============================================================

target_cols = [
    'price', 'positive_ratio', 'total_reviews', 'game_age_years',
    'metacritic_score', 'popularity_score', 'n_platforms',
    'achievements', 'average_playtime_forever', 'dlc_count'
]

# Alternativas por si alguna columna no existe
alt_cols = {
    'total_reviews': ['log_total_reviews'],
    'n_platforms': ['windows', 'mac', 'linux'],
    'achievements': ['achievements'],
    'dlc_count': ['dlc_count']
}

available_cols = []
for col in target_cols:
    if col in df.columns:
        available_cols.append(col)
    else:
        for alt in alt_cols.get(col, []):
            if alt in df.columns and alt not in available_cols:
                available_cols.append(alt)
                break

print(f'Columnas para correlacion: {available_cols}')

df_corr = df[available_cols].copy()
# Convertir booleanos a int
for c in df_corr.columns:
    if df_corr[c].dtype == bool:
        df_corr[c] = df_corr[c].astype(int)
    elif df_corr[c].dtype == object:
        df_corr[c] = pd.to_numeric(df_corr[c], errors='coerce')

corr_matrix = df_corr.corr()

fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt='.2f',
    cmap='RdBu_r',
    center=0,
    vmin=-1, vmax=1,
    square=True,
    linewidths=0.5,
    ax=ax,
    cbar_kws={'shrink': 0.8}
)
ax.set_title('Correlaciones entre Variables Numéricas Clave\n(triángulo inferior)', fontsize=14, fontweight='bold')
plt.tight_layout()

out_corr = figures_dir / 'reporte_correlaciones.png'
fig.savefig(str(out_corr), dpi=150, bbox_inches='tight')
plt.show()
print(f'Figura guardada: {out_corr}')

Columnas para correlacion: ['price', 'positive_ratio', 'total_reviews', 'game_age_years', 'metacritic_score', 'popularity_score', 'n_platforms', 'achievements', 'average_playtime_forever', 'dlc_count']


Figura guardada: C:\Users\Christian Ruiz\Maestria_DS\Gestion_Datos\reports\figures\reporte_correlaciones.png


## 6. Segmentación por Precio

Análisis de la distribución y calidad percibida por segmento de precio.

In [8]:
# ============================================================
# CELDA 14 — Análisis por Price Tier
# ============================================================

price_tier_col = 'price_tier' if 'price_tier' in df.columns else None

if price_tier_col is None and 'price' in df.columns:
    # Crear price_tier manualmente
    df_plot = df.copy()
    conditions = [
        df_plot['price'] == 0,
        df_plot['price'] <= 5,
        df_plot['price'] <= 15,
        df_plot['price'] <= 30,
        df_plot['price'] > 30
    ]
    choices = ['Gratis', '$0.01-$5', '$5.01-$15', '$15.01-$30', '$30+']
    df_plot['price_tier'] = np.select(conditions, choices, default='Desconocido')
    price_tier_col = 'price_tier'
else:
    df_plot = df.copy()

if price_tier_col and price_tier_col in df_plot.columns:
    tier_order_full = ['Gratis', '$0.01-$5', '$5.01-$15', '$15.01-$30', '$30+', 
                       'free', 'budget', 'mid', 'premium', 'luxury']
    existing_tiers = df_plot[price_tier_col].value_counts().index.tolist()
    tier_order = [t for t in tier_order_full if t in existing_tiers]
    tier_order += [t for t in existing_tiers if t not in tier_order_full]

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Análisis por Segmento de Precio', fontsize=15, fontweight='bold')

    # Barplot: count por tier
    tier_counts = df_plot[price_tier_col].value_counts()
    tier_counts_ordered = tier_counts.reindex(
        [t for t in tier_order if t in tier_counts.index]
    ).dropna()

    colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c', '#9b59b6']
    bars = axes[0].bar(
        range(len(tier_counts_ordered)),
        tier_counts_ordered.values,
        color=colors[:len(tier_counts_ordered)],
        edgecolor='white',
        alpha=0.85
    )
    axes[0].set_xticks(range(len(tier_counts_ordered)))
    axes[0].set_xticklabels(tier_counts_ordered.index, rotation=30, ha='right', fontsize=10)
    axes[0].set_ylabel('Número de Juegos', fontsize=11)
    axes[0].set_title('Distribución por Segmento de Precio', fontsize=12)
    for bar, val in zip(bars, tier_counts_ordered.values):
        axes[0].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 50,
                     f'{val:,}', ha='center', va='bottom', fontsize=9)

    # Boxplot: positive_ratio por tier
    if 'positive_ratio' in df_plot.columns:
        tier_data = [
            df_plot[df_plot[price_tier_col] == tier]['positive_ratio'].dropna() * 100
            for tier in tier_counts_ordered.index
        ]
        bp = axes[1].boxplot(
            tier_data,
            labels=tier_counts_ordered.index,
            patch_artist=True,
            notch=False
        )
        for patch, color in zip(bp['boxes'], colors[:len(tier_counts_ordered)]):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        axes[1].set_xlabel('Segmento de Precio', fontsize=11)
        axes[1].set_ylabel('Ratio de Reviews Positivas (%)', fontsize=11)
        axes[1].set_title('Calidad Percibida por Segmento de Precio', fontsize=12)
        axes[1].tick_params(axis='x', rotation=30)
    else:
        axes[1].text(0.5, 0.5, 'positive_ratio no disponible',
                     ha='center', va='center', transform=axes[1].transAxes)

    plt.tight_layout()
    out_tier = figures_dir / 'reporte_precio_tier.png'
    fig.savefig(str(out_tier), dpi=150, bbox_inches='tight')
    plt.show()
    print(f'Figura guardada: {out_tier}')
else:
    print('Columna price_tier no disponible.')

Figura guardada: C:\Users\Christian Ruiz\Maestria_DS\Gestion_Datos\reports\figures\reporte_precio_tier.png


## 7. Variables Target para Modelado

Análisis de las variables candidatas a ser target en modelos predictivos, incluyendo su distribución y características estadísticas.

In [9]:
# ============================================================
# CELDA 16 — Análisis de Variables Target
# ============================================================
from scipy import stats as scipy_stats

candidate_targets = [
    ('positive_ratio', 'Regresion', 'Ratio de reviews positivas'),
    ('popularity_score', 'Regresion', 'Score de popularidad'),
    ('log_total_reviews', 'Regresion', 'Log de total de reviews'),
    ('is_free', 'Clasificacion', 'Indicador de juego gratuito'),
    ('price_tier', 'Clasificacion multiclase', 'Segmento de precio')
]

target_rows = []
for col, tipo, desc in candidate_targets:
    alt = None
    if col not in df.columns:
        if col == 'log_total_reviews' and 'total_reviews' in df.columns:
            df['log_total_reviews'] = np.log1p(df['total_reviews'])
            alt = 'calculada'
        elif col == 'price_tier' and 'price' in df.columns:
            # Ya creado en celda anterior si df_plot existe, pero re-crear aqui
            conditions = [
                df['price'] == 0,
                df['price'] <= 5,
                df['price'] <= 15,
                df['price'] <= 30,
                df['price'] > 30
            ]
            choices = ['Gratis', '$0.01-$5', '$5.01-$15', '$15.01-$30', '$30+']
            df['price_tier'] = np.select(conditions, choices, default='Desconocido')
            alt = 'calculada'
        else:
            target_rows.append({
                'Variable': col,
                'Tipo': tipo,
                'Descripcion': desc,
                'Skewness': 'N/A',
                'Kurtosis': 'N/A',
                '% Nulos': 'N/A',
                'Rango': 'N/A',
                'Estado': 'NO DISPONIBLE'
            })
            continue
    
    serie = df[col]
    pct_nulos = serie.isna().mean() * 100
    
    if tipo in ['Regresion'] or serie.dtype in [np.float64, np.float32, np.int64, np.int32]:
        try:
            s_num = pd.to_numeric(serie, errors='coerce').dropna()
            skew_val = f'{s_num.skew():.2f}'
            kurt_val = f'{s_num.kurtosis():.2f}'
            rango = f'[{s_num.min():.3f}, {s_num.max():.3f}]'
        except Exception:
            skew_val = 'N/A'
            kurt_val = 'N/A'
            rango = 'N/A'
    else:
        skew_val = 'N/A (categorica)'
        kurt_val = 'N/A (categorica)'
        n_cats = serie.nunique()
        rango = f'{n_cats} categorias: {list(serie.dropna().unique()[:3])}...'
    
    estado = 'disponible' if col in df.columns else f'calculada ({alt})'
    
    target_rows.append({
        'Variable': col,
        'Tipo': tipo,
        'Descripcion': desc,
        'Skewness': skew_val,
        'Kurtosis': kurt_val,
        '% Nulos': f'{pct_nulos:.1f}%',
        'Rango': rango,
        'Estado': estado
    })

df_targets = pd.DataFrame(target_rows)
print('Variables Candidatas a Target para Modelado:')
display(df_targets.style.hide(axis='index'))

Variables Candidatas a Target para Modelado:


Variable,Tipo,Descripcion,Skewness,Kurtosis,% Nulos,Rango,Estado
positive_ratio,Regresion,Ratio de reviews positivas,-1.33,1.57,32.3%,"[0.000, 1.000]",disponible
popularity_score,Regresion,Score de popularidad,-0.35,0.15,0.0%,"[0.000, 0.670]",disponible
log_total_reviews,Regresion,Log de total de reviews,0.96,0.56,0.0%,"[0.000, 15.992]",disponible
is_free,Clasificacion,Indicador de juego gratuito,1.40,-0.05,0.0%,"[0.000, 1.000]",disponible
price_tier,Clasificacion multiclase,Segmento de precio,N/A (categorica),N/A (categorica),0.0%,"6 categorias: ['Free', 'Mid ($5-15)', 'Low (<$5)']...",disponible


## 8. Recomendaciones para Modelado

Propuestas de modelos predictivos basadas en los hallazgos del EDA.

In [10]:
# ============================================================
# CELDA 18 — Tabla de Recomendaciones de Modelado
# ============================================================

rec_data = [
    {
        'Modelo': 'Random Forest Regressor',
        'Target': 'positive_ratio',
        'Features Clave': 'genre_primary, game_age_years, price, n_dlc, log_total_reviews',
        'Baseline Esperado': 'MAE < 0.10, R2 > 0.50',
        'Justificacion': 'Alta interpretabilidad; maneja features categoricas y numericas mixtas'
    },
    {
        'Modelo': 'XGBoost Regressor',
        'Target': 'log_total_reviews (popularidad)',
        'Features Clave': 'genre_primary, price_tier, era, is_free, game_age_years',
        'Baseline Esperado': 'RMSE < 1.5, R2 > 0.60',
        'Justificacion': 'Mejor rendimiento en datos tabulares; maneja outliers'
    },
    {
        'Modelo': 'Logistic Regression',
        'Target': 'is_free (binario)',
        'Features Clave': 'genre_primary, developer_freq, publisher_freq, n_platforms',
        'Baseline Esperado': 'AUC-ROC > 0.85, F1 > 0.80',
        'Justificacion': 'Linea base interpretable para clasificacion binaria'
    },
    {
        'Modelo': 'LightGBM Classifier',
        'Target': 'price_tier (multiclase)',
        'Features Clave': 'genre_primary, era, developer_freq, metacritic_score, n_platforms',
        'Baseline Esperado': 'Accuracy > 0.70, Macro-F1 > 0.60',
        'Justificacion': 'Eficiente con alta cardinalidad; natural para multiclase'
    },
    {
        'Modelo': 'KMeans Clustering',
        'Target': 'Segmentacion (no supervisado)',
        'Features Clave': 'price, positive_ratio, log_total_reviews, game_age_years, genre_encoded',
        'Baseline Esperado': 'Silhouette Score > 0.35',
        'Justificacion': 'Identificar arquetipos de juegos para estrategia de pricing'
    }
]

df_rec = pd.DataFrame(rec_data)
print('Recomendaciones de Modelado Predictivo:')
display(df_rec.style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#2c3e50'), ('color', 'white'), ('font-size', '12px')]},
    {'selector': 'td', 'props': [('font-size', '11px'), ('padding', '5px 10px'), ('vertical-align', 'top')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#eaf2ff')]}
]).hide(axis='index'))

Recomendaciones de Modelado Predictivo:


Modelo,Target,Features Clave,Baseline Esperado,Justificacion
Random Forest Regressor,positive_ratio,"genre_primary, game_age_years, price, n_dlc, log_total_reviews","MAE < 0.10, R2 > 0.50",Alta interpretabilidad; maneja features categoricas y numericas mixtas
XGBoost Regressor,log_total_reviews (popularidad),"genre_primary, price_tier, era, is_free, game_age_years","RMSE < 1.5, R2 > 0.60",Mejor rendimiento en datos tabulares; maneja outliers
Logistic Regression,is_free (binario),"genre_primary, developer_freq, publisher_freq, n_platforms","AUC-ROC > 0.85, F1 > 0.80",Linea base interpretable para clasificacion binaria
LightGBM Classifier,price_tier (multiclase),"genre_primary, era, developer_freq, metacritic_score, n_platforms","Accuracy > 0.70, Macro-F1 > 0.60",Eficiente con alta cardinalidad; natural para multiclase
KMeans Clustering,Segmentacion (no supervisado),"price, positive_ratio, log_total_reviews, game_age_years, genre_encoded",Silhouette Score > 0.35,Identificar arquetipos de juegos para estrategia de pricing


## 9. Pipeline de Archivos Generados

Inventario completo de artefactos del análisis exploratorio.

In [11]:
# ============================================================
# CELDA 20 — Inventario de Archivos Generados
# ============================================================
import os

def listar_archivos(directorio, extensiones=None):
    """Lista archivos en un directorio con su tamaño en MB."""
    dir_path = PROJECT_ROOT / directorio
    filas = []
    if not dir_path.exists():
        return filas
    for archivo in sorted(dir_path.iterdir()):
        if archivo.is_file():
            if extensiones and archivo.suffix.lower() not in extensiones:
                continue
            tamanio_mb = archivo.stat().st_size / (1024 * 1024)
            filas.append({
                'Directorio': directorio,
                'Archivo': archivo.name,
                'Extension': archivo.suffix,
                'Tamanio (MB)': f'{tamanio_mb:.2f}'
            })
    return filas

todos_archivos = []
todos_archivos += listar_archivos('data/processed', ['.csv', '.parquet'])
todos_archivos += listar_archivos('reports/figures', ['.png', '.svg'])
todos_archivos += listar_archivos('reports/tables', ['.md', '.csv', '.html'])
todos_archivos += listar_archivos('notebooks/exploratory', ['.ipynb'])
todos_archivos += listar_archivos('notebooks/reports', ['.ipynb'])

if todos_archivos:
    df_inv = pd.DataFrame(todos_archivos)
    total_mb = df_inv['Tamanio (MB)'].astype(float).sum()
    print(f'Total de archivos encontrados: {len(todos_archivos)} | Tamanio total: {total_mb:.1f} MB')
    display(df_inv.style.hide(axis='index'))
else:
    print('No se encontraron archivos en los directorios especificados.')

Total de archivos encontrados: 25 | Tamanio total: 217.0 MB


Directorio,Archivo,Extension,Tamanio (MB)
data/processed,steam_games_clean_2026-02-11.csv,.csv,77.30
data/processed,steam_games_clean_2026-02-11.parquet,.parquet,32.49
data/processed,steam_games_features_2026-02-12.csv,.csv,104.21
reports/figures,distribucion_features_derivadas.png,.png,0.19
reports/figures,heatmap_nulos.png,.png,0.03
reports/figures,reporte_correlaciones.png,.png,0.13
reports/figures,reporte_distribuciones.png,.png,0.14
reports/figures,reporte_precio_tier.png,.png,0.11
reports/figures,viz10_boxplots_genero.png,.png,0.27
reports/figures,viz11_evolucion_precio_anio.png,.png,0.14


## 10. Conclusiones

### Hallazgos Estructurales del Dataset

El dataset de Steam Games revela un mercado altamente asimétrico y fragmentado. La plataforma alberga más de 122,000 títulos, pero la distribución de visibilidad y éxito sigue una ley de potencia pronunciada: una minoría de juegos concentra la mayoría de reviews y engagement, mientras que la gran mayoría permanece virtualmente invisible.

### Factores Predictivos Identificados

**El ratio de reviews positivas** emerge como la variable de calidad más robusta y bien distribuida para modelado. Su mediana supera el 80%, con distribución bimodal que separa juegos bien recibidos de aquellos que generan reacciones negativas intensas.

**El género primario** es el factor categórico más informativo: Indie, Action y Adventure dominan por volumen, pero los géneros de nicho (Simulation, RPG, Strategy) exhiben mejor ratio precio-calidad percibida.

**La era de lanzamiento** captura tendencias históricas de mercado. El período 2014-2018 (era Greenlight/Direct) concentra el mayor volumen con mayor varianza en calidad, mientras que los juegos más recientes y los clásicos pre-2010 muestran ratios positivos superiores.

### Implicaciones para el Negocio

1. **Estrategia de pricing**: El mercado de pago está concentrado bajo USD 15. Juegos por encima de USD 30 representan menos del 10% pero capturan ratios de calidad comparables, sugiriendo disposición a pagar por productos premium bien valorados.

2. **Modelo de distribución**: El 98%+ de soporte exclusivo a Windows limita el potencial de diferenciación por plataforma como variable predictiva, pero el soporte multiplataforma (Mac + Linux) correlaciona positivamente con el ratio de aprobación.

3. **Discovery y visibilidad**: La correlación entre total de reviews y ratio positivo sugiere que los mecanismos de recomendación de Steam crean un sesgo hacia títulos ya populares. Un modelo de "cold start" es crítico para nuevos lanzamientos.

### Próximos Pasos Recomendados

- **Fase 1**: Entrenar modelos baseline (Regresión Lineal, Random Forest) sobre `positive_ratio` y `log_total_reviews`.
- **Fase 2**: Incorporar features de texto (descripción del juego, tags TF-IDF) para mejorar la representación semántica.
- **Fase 3**: Diseñar sistema de recomendación basado en colaboración implícita usando patrones de co-compra y reviews.
- **Fase 4**: Implementar monitoreo de drift para detectar cambios en el mercado de videojuegos a lo largo del tiempo.

---

*Análisis completado el 2026-02-12. Dataset fuente: Steam Games Dataset (Kaggle - fronkongames). Notebooks EDA disponibles en `notebooks/exploratory/`.*